## Homework 2: Search hyperparameters for neural networks
In this homework, you will practice using Keras to implement neural networks and search their hyperparameters for handwritten digits classification.

### Dataset
MNIST handwritten digits. 60k 28*28 grayscale training images of the 10 digits, along with a test set of 10k images

![Fig. 1](handwrittendigits.png)
 *Fig.1. Handwritten digits examples.* 

### Instructions

    1. You need to install Keras on your computer to build neural networks.
    2. The framework, e.g., functions' names, input, and output, has been defined. You are going to complete the create_NN, nn_params_search, retrain_best_nn, myEvaluation functions.
    3. Add your code in the following blocks, and do not change other places.

```python

    ## add your code here
    
    ##
```

### Student information
    1. Your name: Carlos Santos
    2. Department: Comp. E.
    3. Undergraduate

### Task points and TA grading: ??/100
    1. ?/10
    2. ?/30
    3. ?/30
    4. ?/10
    5. ?/20
    6: performance_acc: ?/10

In [4]:
import keras
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import numpy as np

### 1. Load the MNIST dataset in Keras. 10 points

In [5]:
import matplotlib.pyplot as plt
def load_data():
    '''Load the MNIST dataset'''
    
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
    return (X_train, y_train, X_test, y_test)

#Task 1. load the dataset
(X_train_2D, y_train, X_test_2D, y_test) = load_data()

# 1.1 reshape the training and test sets (N*28*28) to N * 784. 10 points
## add your code here
X_train = 
X_test = 
print('X_train_2D: {}, X_train: {}'.format(X_train_2D.shape, X_train.shape))
print('X_test_2D: {}, X_test: {}'.format(X_test_2D.shape, X_test.shape))
##

# 1.2 transform y_train to one-hot vectors using keras.utils.to_categorical to form the output of the NN. 10 points
## add your code here
y_train_onehot = 
print('y_train_onehot: {}'.format(y_train_onehot.shape))
print('one-hot vector examples:\n', y_train_onehot[:5])
##

SyntaxError: invalid syntax (<ipython-input-5-33f063e72947>, line 13)

### 2. Create neural networks using Keras. 30 points

In [ ]:
def create_NN(hidden_layers = [1000], act = 'relu', opt = 'rmsprop'): 
    '''create a deep feedforwad neural network using keras
    
    Parameters
    -----------
    hidden_layers: a list that defines the numbers of hidden nodes for all hidden layers, e.g., [1000] indicates
    the nn has only one hidden layer with 1000 nodes, while [1000, 500] defines two hidden layers and the first
    layer has 1000 nodes and the second has 500 nodes.
    act: activation function for all hidden layers
    opt: optimizer
    
    Returns
    -------
    myNN: the neural network model
    
    '''
    in_dim = 784
    out_dim = 10
    
    ## add your code here
    myNN = keras.models.Sequential()
    
    #2.1 build all hidden layers

        
        
    #2.2 build the output layer and use the softmax activation  

    
    
    #2.3 choose the optimizer, compile the network and return it. Use 'accuracy' as the metrics

    
    
    return myNN

    ##
    
h_nodes = [1000, 500] # two hidden layers with 1000 and 500 nodes, respectively.
myNN = create_NN(hidden_layers= h_nodes, act = 'relu', opt = 'adam')
myNN.summary()

### 3. Search the best NN paprameters, and report the performance. 30 points
    - The KerasClassifier will be used to warp NN models to use the GridSearchCV
    - Complete the nn_params_search function to search the three parameters: batch_size, activation, and optimizer.
    - Each fit (10 epochs) may take 1 to 2 minutes.

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

def nn_params_search(nn, X, y, param_grid): # 30 points
    '''Search best paramaters
    
    Parameters
    ----------
    X_train: features
    y_train: target of the input
    param_grid: a dict that defines the parameters
    
    Returns
    -------
    best_params_
        
    '''
    ## add your code here. set cv = 3, scoring = 'accuracy', and verbose = 2
    
    
    
    ##

# wrap keras model to use in sklearn
nn = KerasClassifier(build_fn = create_NN, batch_size = 64, epochs = 10) # using the keras wapper
param_grid = {'batch_size': [64, 128], 
              'act':['relu', 'sigmoid'], 'opt': ['sgd', 'adam']}

best_params = nn_params_search(nn, X_train, y_train, param_grid = param_grid)
print('\nBest parameters: ', best_params)

### 4. Retrain a neural network using the best parameters. 10 points
    - Compelte the retrain_test_nn function to create (create_NN) and train (fit) a new nn using parameters in best_params
    - The default epoches are 20 and each epoch may take around 10 to 20 seconds; and you can increase this value to get better results

In [6]:
def retrain_best_nn(best_params, X_train, y_train, epochs = 10): # 10 points
    '''Retrain a nn using the best parameters
    
    Paramters
    ----------
    best_params:
    X_train: data input of the training set
    y_train: target of the input (one-hot vectors)
    
    Returns
    ---------
    bestNN: the nn classifier trained using the best parameters
    
    '''
    ## add your code here  

    
    
    
    ##
#
bestNN = retrain_best_nn(best_params, X_train, y_train_onehot, epochs = 20)

NameError: name 'best_params' is not defined

### 5. Network evaluation. 20 points

    - Complete the myEvaluation function to report the performance of your best nn using the test set
        - compute the overall accuracy
        - compute the precision for each class

In [ ]:
from sklearn.metrics import classification_report

def myEvaluation(y, y_pred):
    ''' calculate the overall accuracy and precision
    
        Parameters
        ----------
        y: real target
        y_pred: prediction
        
        Returns
        -------
        acc: accuracy
        precision: precision array
    '''
    ## add your code here
    # calculate the overall acc

    
    # calculate the precision for each class

    
    # return acc and precision array
    
    
    ##

#
y_test_pred = bestNN.predict_classes(X_test)
acc, precision = myEvaluation(y_test, y_test_pred)
print('my accuracy:    {:.2f}'.format(acc))
print('my precision:', precision)

# results calculated using the classification_report
print(classification_report(y_test, y_test_pred))

### 6. Any findings or conclusion, e.g., useful strategies to improve nn performance. Extra 10 points
1.

2.

3.

4....
